In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=0a7a9ecc54a3f45c56e76ea556edfd9d15a8a11af8ba40c801c84aa3a09f30bd
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
#import pyLDAvis
import re
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from nltk.corpus import stopwords
import multiprocessing as mps
import multiprocess as mp
from joblib import Parallel, delayed
import io
from google.colab import files
import functools
from itertools import repeat
import gensim

In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['AAPL.csv']))

In [ ]:
url = 'https://raw.githubusercontent.com/weirdaxe/dissertation/main/trial_data/AAPL.csv?token=GHSAT0AAAAAABTR7NYFMWEC4UVUXF4NI6O4YSYB37Q'
df = pd.read_csv(url)

HTTPError: ignored

In [ ]:
''' Get stopwords and rudementary clean of the text data'''

def get_stopwords():
    stop_words = stopwords.words("english")
    stopwords_set = set(stop_words)
    return stopwords_set

def cleaner(df):
    pattern = re.compile(r"[A-Za-z0-9\-]{3,50}")
    df['clean'] = df['content'].str.findall(pattern).str.join(' ')
    return df

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = get_stopwords()
df_preproc = cleaner(df)

In [ ]:
nlp = spacy.load("en_core_web_sm",disable=["ner"])

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [ ]:
def merge_phrases(doc):
    with doc.retokenize() as retokenizer:
        for np in list(doc.noun_chunks):
            attrs = {
                "tag": np.root.tag_,
                "lemma": np.root.lemma_,
                "ent_type": np.root.ent_type_,
            }
            retokenizer.merge(np, attrs=attrs)
    return doc

In [ ]:
pos_list = ["VERB","NOUN","ADJ","ADV","PROPN"]
lemma_list = []
doc = nlp("Dow Jones the industrial complex is down 10%")
#for noun_phrase in list(doc.noun_chunks):
#    noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
for token in doc:
    child = len([child for child in token.children])
    alpha = token.is_alpha
    text = token.text.lower()
    pos = token.pos_
    if alpha and text not in stop_words and pos in pos_list:
      if child==0:
        lemma_list.append(str(token.lemma_).lower())
      else:
        lemma_list.extend(repeat(str(token.lemma_).lower(),child))
    #elif alpha and text not in stop_words and pos not in pos_list and child!=0:
    #    lemma_list.extend(repeat(str(token.lemma_).lower(),child))

In [ ]:
doc = nlp("Dow Jones the industrial complex is down 10%")
for noun_phrase in list(doc.noun_chunks):
  noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
  print(noun_phrase.text)

Dow Jones
the industrial complex


In [ ]:
for t in doc:
  print(t.text,t.pos_)

Dow Jones PROPN
the industrial complex NOUN
is AUX
down ADV
10 NUM
% NOUN


In [ ]:
def bigram_phrase (data_words):
    bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold = 50)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    return bigram, bigram_phrases

def trigram_phrase (data_words, bigram_phrases):
    trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)
    return trigram
    
def make_bigrams(text, bigram):
    return [bigram[doc] for doc in text]

def make_trigrams(text, trigram):
    return [trigram[doc] for doc in text]

def data_bigrams_trigrams (data_words):
    bigram, bigram_phrases = bigram_phrase(data_words)
    trigram = trigram_phrase (data_words, bigram_phrases)
    data_bigrams = make_bigrams(data_words, bigram)
    data_bigrams_trigrams = make_trigrams(data_bigrams, trigram)
    return data_bigrams_trigrams

In [ ]:
''' Define functions for extracting the lemma from doc and itterate through the document'''

def lemma_list(doc):
    pos_list = ["VERB","NOUN","ADJ","ADV","PROPN"]
    lemma_list = []
    for token in doc:
      child = len([child for child in token.children])
      alpha = token.is_alpha
      text = token.text.lower()
      pos = token.pos_
      if alpha and text not in stop_words and pos in pos_list:
        #lemma_list.append(str(token.lemma_).lower())
        if child==0:
          lemma_list.append(str(token.lemma_).lower())
        else:
          lemma_list.extend(repeat(str(token.lemma_).lower(),child))  
    return lemma_list

def create_doc(article, nlp):
    doc = nlp(article)
    return lemma_list(doc)

def create_doc_g(article):
    doc = nlp(article)
    return lemma_list(doc)

def lemma_data(df):
    nlp = spacy.load("en_core_web_sm",disable=["ner"])
    cd_mid = functools.partial(create_doc, nlp=nlp)
    pool = mp.Pool(mp.cpu_count())
    results = pool.map(cd_mid, [a for a in df['clean']])
    pool.close()
    return results

def lemma_data_joblib(df):
  nlp = spacy.load("en_core_web_sm",disable=["ner"])
  cd_mid = functools.partial(create_doc, nlp=nlp)
  return Parallel(n_jobs=-1, prefer="threads",
                   verbose=0)(delayed(cd_mid)(a) for a in df['clean'])

def lemma_gpu(df):
  nlp = spacy.load("en_core_web_sm",disable=["ner"])
  result = []
  for doc in nlp.pipe(df["clean"]):
    result.append(lemma_list(doc))
  return result

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lemma_data(df_preproc.loc[:100,:])

In [ ]:
%timeit lemma_data(df_preproc.loc[:100,:])

1 loop, best of 5: 29.4 s per loop


In [ ]:
%timeit lemma_data_joblib(df_preproc.loc[:100,:])

1 loop, best of 5: 11.4 s per loop


In [ ]:
spacy.require_gpu()

True

In [ ]:
!nvidia-smi

Thu Apr 14 11:11:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%timeit lemma_gpu(df_preproc.loc[0:1000,:])

1 loop, best of 5: 11.8 s per loop


In [ ]:
df_preproc['lemma'] = lemma_gpu(df_preproc)

In [ ]:
from joblib import Parallel, delayed

def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stop_words] 
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemma_list(doc))
    return preproc_pipe

def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemma_list(doc))
    return preproc_pipe

def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=mp.cpu_count(), backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(df_preproc), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['ner'])
a = preprocess_parallel(df_preproc.loc[:100,'clean'], chunksize=1000)

In [ ]:
%timeit preprocess_parallel(df_preproc.loc[:1000,'clean'],chunksize=1000)

1 loop, best of 5: 34.5 s per loop


In [ ]:
lda_df = data_bigrams_trigrams(df_preproc['lemma'])

In [ ]:
df_preproc.head()

In [ ]:
df_preproc[df_preproc['lemma'].map(lambda d: len(d)) == 0]

,Unnamed: 0,date,title,content,link,symbols,tags,clean,lemma


In [ ]:
'''LDA Model'''
dictionary = gensim.corpora.Dictionary(df_preproc['lemma'])
dictionary.filter_extremes(no_below=5, no_above=0.7)
corpus = [dictionary.doc2bow(doc) for doc in df_preproc['lemma']]
id2word = dictionary.id2token
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8246
Number of documents: 3000


In [ ]:
lda_model = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus[:-1],
    id2word=dictionary,
    num_topics=30,
    random_state=100,
    #update_every=1,
    chunksize=1000,
    passes=10,
    workers = 2)
#alpha="auto")

In [ ]:
corpus[1893]

In [ ]:
dictionary[1854]

'marketable'

In [ ]:
topics[1893]

In [ ]:
for i in topics[1890:1900]:
  print (i)

In [ ]:
a=[]
for i in range (len(topics)-1):
  if i%100==0:
    print(i)
  a.append(topics[i])

In [ ]:
a

In [ ]:
topics = lda_model.get_document_topics(corpus)
a = []
for doc in topics:
  a.append(doc)
all_topics = [(doc_topics, phi) for doc_topics, phi in topics]

In [ ]:
def get_df_topics(lda,corpus):
  topic = []
  all_topics = lda.get_document_topics(corpus)
  for doc_topics in all_topics:
    topic.append(doc_topics)
  return topic
a = get_df_topics(lda=lda_model, corpus=corpus)

In [ ]:
import pyLDAvis.gensim_models

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model,corpus, dictionary, mds='mmds')
pyLDAvis.display(vis)

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model,corpus, dictionary, mds='mmds')
pyLDAvis.display(vis)

In [ ]:
!pip install lda2vec

  Created wheel for lda2vec: filename=lda2vec-0.16.10-py3-none-any.whl size=14433 sha256=82b989c9b8125c97c02fe6e91ac1638487fc6ce4729e3ab6683229156c69e242
  Stored in directory: /root/.cache/pip/wheels/76/51/1c/1e2960ded8d6e14b14454493fcc9c2d879a7fb99421867f177
Successfully built lda2vec


In [ ]:
import lda2vec
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
a[0]

[(13, 0.3723632), (21, 0.5887479)]

In [ ]:
df1 = pd.DataFrame(df_preproc['lemma'])

In [ ]:
df_preproc['topics']=a

In [ ]:
df1.head()

,lemma,topics
0,"[dow, jones, future, signal, market, rally, ra...","[(13, 0.3723632), (21, 0.5887479)]"
1,"[article, first, appear, appear, appear, morni...","[(2, 0.045572024), (3, 0.04024079), (6, 0.1367..."
2,"[dow, jones, future, signal, market, rally, ra...","[(13, 0.37245306), (21, 0.5886581)]"
3,"[west, ramp, ramp, sanction, sanction, sanctio...","[(16, 0.17022663), (21, 0.78310674)]"
4,"[berkshire, hathaway, brk, beat, beat, beat, e...","[(4, 0.26927215), (15, 0.52440566), (28, 0.167..."


In [ ]:
df2 = pd.DataFrame(a).fillna(0)
#df2 = df2.fillna(0)

In [ ]:
def topics_to_df (df):
  df_end = pd.DataFrame()

  for i in range(30):
      df_end["top_"+str(i)]=[]

  frames = [df["topics"],df_end]
  df_end = pd.concat(frames).fillna(0)

  for i in range(len(df['topics'])):
      for j in range(len(df['topics'][i])):
          df_end["top_"+str(df['topics'][i][j][0])].loc[i] = df['topics'][i][j][1]
  return df.merge(df_end, left_index=True, right_index=True).drop(0,axis=1)#.rename(columns={"topics_x":"topics"})

In [ ]:
df2 = topics_to_df(df_preproc)

In [ ]:
df2.loc[0]

Unnamed: 0                                                    0
date                                  2022-02-28T13:07:17+00:00
title         Dow Jones Futures Pare 'Swift' Market Losses; ...
content       Dow Jones futures signal a market rally revers...
link          https://finance.yahoo.com/m/5e9706a2-d66c-3d05...
symbols       ['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...
tags          ['DOW JONES', 'STOCK MARKET', 'NASDAQ FUTURES'...
clean         Dow Jones futures signal market rally reversal...
lemma         [dow, jones, future, signal, market, rally, ra...
topics                       [(13, 0.3723632), (21, 0.5887479)]
top_0                                                       0.0
top_1                                                       0.0
top_10                                                      0.0
top_11                                                      0.0
top_12                                                      0.0
top_13                                  

In [ ]:
!nvidia-smi

Thu Apr 21 09:38:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 880 kB 40.3 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 23.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=803d671dfe03ce0b3542d7b9426d48a8312ddc336b24b290eb7cf5813ea4c419
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
import gc

device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer
from bertModel import BertClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AAPL_topics.csv'
df_bonus = pd.read_csv(path)

In [ ]:
finbert_list_title = df_bonus['title'].to_list()

In [ ]:
inputs = tokenizer(df_bonus["title"][0], padding = True, truncation = True, return_tensors='pt')

In [ ]:
inputs

{'input_ids': tensor([[  101, 23268,  3557, 17795, 11968,  2063,  1005,  9170,  1005,  3006,
          6409,  1025,  2225,  7889,  3607,  5085, 13463,  5924,  5274,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
model.to("cpu")

In [ ]:
finbert_list_title[2998:3000]

['Apple’s Troubles Are About Supply, Not Demand. Why Analysts Are Still Upbeat.',
 '3 Disruptive Stocks Shaping the Future of Technology']

In [ ]:
def sentiment_sc(text):
  inputs = tokenizer(text, padding = True, truncation = True, return_tensors='pt')
  outputs = model(**inputs)
  pred = torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu").tolist()
  # del outputs, inputs
  # gc.collect()
  # torch.cuda.empty_cache()
  return pred

In [ ]:
!nvidia-smi

Tue May  3 12:19:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    71W / 149W |    956MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df_bonus["clean"][:10]

0    Dow Jones futures signal market rally reversal...
1    This article first appeared the Morning Brief ...
2    Dow Jones futures signal market rally reversal...
3    The West ramps sanctions Russia amid the ongoi...
4    Berkshire Hathaway BRK beat estimates for earn...
5    Adds quote background OSLO Feb Reuters Norway ...
6    With both the 500 and Nasdaq Composite correct...
7    this clip from The Future Fintech Motley Fool ...
8    This motley collection companies intriguing mi...
9    this article discuss the best small-cap stocks...
Name: clean, dtype: object

In [ ]:
inputs = tokenizer(finbert_list_title[a:i], padding = True, truncation = True, return_tensors='pt').to(device)

In [ ]:
len(c)

4

In [ ]:
cc = c+c2

In [ ]:
input_ids = torch.stack(cc)

In [ ]:
len(list1[3]["attention_mask"])

8

In [ ]:
list1 = []
a=0
len_dict={}
for i in range(4,17,4):
  e = tuple()
  f = tuple()
  for j in range(a,i+1):
    input = tokenizer.encode_plus(df_bonus['clean'][j], return_tensors='pt')
    p = mask_chunk(input, chunksize=512)
    c = input_id_chunk(input, chunksize=512)
    e += c
    f += p
    len_dict[j] = len(c)
  input_ids = torch.stack(e).to(device)
  attention_mask = torch.stack(f).to(device)
  input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()}
  list1.append(input_dict)
  a=i

Token indices sequence length is longer than the specified maximum sequence length for this model (1853 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
result = get_sentiment(df = df_bonus, doc_range=3000, device = device)

In [ ]:
result_flat = [item for sublist in result for item in sublist]

In [ ]:
result_flat

In [ ]:
df_results = pd.DataFrame()

In [ ]:
df_bonus[["positive","negative","neutral"]] = result_flat

In [ ]:
df_bonus.to_csv('df_w_sentiment.csv', encoding = 'utf-8-sig') 
files.download('df_w_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_bonus.head()

,Unnamed: 0,Unnamed: 0.1,date,title,content,link,symbols,tags,clean,lemma,...,top_3,top_4,top_5,top_6,top_7,top_8,top_9,positive,negative,neutral
0,0,0,2022-02-28T13:07:17+00:00,Dow Jones Futures Pare 'Swift' Market Losses; ...,Dow Jones futures signal a market rally revers...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'NASDAQ FUTURES'...",Dow Jones futures signal market rally reversal...,"['dow', 'jones', 'future', 'signal', 'market',...",...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.379782,0.497155,0.123064
1,1,1,2022-02-28T10:12:12+00:00,Corporate America condemns Russia — what that ...,This article first appeared in the Morning Bri...,https://finance.yahoo.com/news/corporate-ameri...,"['TSLA.US', 'AAPL.US', 'CRM.US', 'ROKU.US', 'S...","['RUSSIA', 'MULTINATIONAL COMPANIES', 'UKRAINE']",This article first appeared the Morning Brief ...,"['article', 'first', 'appear', 'appear', 'appe...",...,0.040241,0.000000,0.0,0.136734,0.0,0.0,0.034227,0.030476,0.212374,0.757150
2,2,2,2022-02-27T23:22:17+00:00,Dow Jones Futures Signal 'Swift' Market Revers...,Dow Jones futures signal a market rally revers...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'RUSSIA', 'DOW J...",Dow Jones futures signal market rally reversal...,"['dow', 'jones', 'future', 'signal', 'market',...",...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.379782,0.497155,0.123064
3,3,3,2022-02-27T18:18:17+00:00,Dow Jones Futures Loom: Watch Out For 'Swift' ...,The West ramps up sanctions vs. Russia amid th...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'NASDAQ FUTURES'...",The West ramps sanctions Russia amid the ongoi...,"['west', 'ramp', 'ramp', 'sanction', 'sanction...",...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.172859,0.428728,0.398414
4,4,4,2022-02-27T16:53:33+00:00,Berkshire Hathaway Earnings: What Happened wit...,Berkshire Hathaway (BRK.A) beat estimates for ...,https://finance.yahoo.com/m/a3289340-767e-3109...,"['BRK-A.US', 'BRK-B.US', 'AAPL.US', 'AAPL34.SA...","['WARREN BUFFETT', 'BERKSHIRE HATHAWAY', 'EARN...",Berkshire Hathaway BRK beat estimates for earn...,"['berkshire', 'hathaway', 'brk', 'beat', 'beat...",...,0.000000,0.269272,0.0,0.000000,0.0,0.0,0.000000,0.937052,0.024245,0.038703


In [ ]:
def get_sentiment(df, chunksize=512, device=device, input_size = 5, doc_range=3000):
  a=0
  pred = []
  len_dict={}
  for i in range(input_size,doc_range+1,input_size):
    e = tuple()
    f = tuple()
    for j in range(a,i):
      input = tokenizer.encode_plus(df['clean'][j], return_tensors='pt')
      p = mask_chunk(input, chunksize=chunksize)
      c = input_id_chunk(input, chunksize=chunksize)
      e += c
      f += p
      len_dict[j] = len(c)
    input_ids = torch.stack(e).to(device)
    attention_mask = torch.stack(f).to(device)
    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()}
    outputs = model(**input_dict)
    pred.append(torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu").tolist())
    del outputs, input
    gc.collect()
    torch.cuda.empty_cache()
    a=i

  return get_correct_results(pred, len_dict)

In [ ]:
def get_correct_results(pred, len_dict):
  result = []
  a=0
  b=0
  predictions = [item for sublist in pred for item in sublist]
  for i in list(len_dict.values()):
    b+=i
    if b-a ==1:
      result.append(predictions[a:b])
    elif b-a>1:
      result.append([list(np.mean(predictions[a:b],axis=0))])
    a=b
  return result

In [ ]:
list(np.mean(predictions[0:1],axis=0))

[0.3797818720340729, 0.4971546530723572, 0.12306341528892517]

In [ ]:
len(len_dict)

101

In [ ]:
result

In [ ]:
predictions = [item for sublist in pred for item in sublist]

In [ ]:
pred

In [ ]:
arrays = [np.array(x) for x in pred]
v = [np.mean(k,axis=0) for k in arrays]

In [ ]:
df_pred = pd.DataFrame(v,columns=["Posisitve","Negative","Neutral"])

In [ ]:
df_kl = pd.DataFrame()

In [ ]:
df_kl[["Positive","Negative","Neutral"]]=v

In [ ]:
df_kl.head()

,Positive,Negative,Neutral
0,0.379782,0.497155,0.123063
1,0.030476,0.212374,0.757150
2,0.379782,0.497155,0.123063
3,0.172859,0.428728,0.398414
4,0.937052,0.024245,0.038703


In [ ]:
df_pred.head()

,Posisitve,Negative,Neutral
0,0.379782,0.497155,0.123063
1,0.030476,0.212374,0.757150
2,0.379782,0.497155,0.123063
3,0.172859,0.428728,0.398414
4,0.937052,0.024245,0.038703


In [ ]:
np.mean(arrays[1],axis=0)

array([0.03047593, 0.21237389, 0.75715017])

In [ ]:
inputs = tokenizer(finbert_list_title[0:300], padding = True, truncation = True, return_tensors='pt').to(device)
outputs = model(**inputs)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
del outputs, inputs

In [ ]:
a=0
pred = []
for i in range(300,901,300):
  inputs = tokenizer(finbert_list_title[a:i], padding = True, truncation = True, return_tensors='pt').to(device)
  print("i",i)
  print("len inputs", len(inputs["input_ids"]))
  outputs = model(**inputs)
  pred.append(torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu").tolist())
  del outputs, inputs
  gc.collect()
  torch.cuda.empty_cache()
  a=i

i 300
len inputs 300
i 600
len inputs 300
i 900
len inputs 300


In [ ]:
pred

In [ ]:
"inputs = tokenizer(finbert_list_title[:300], padding = True, truncation = True, return_tensors='pt').to(device)
#inputs.to(device)

In [ ]:
[1]*10**10

In [ ]:
import gc

In [ ]:
with torch.cuda.device('cuda'):
    torch.cuda.empty_cache()

In [ ]:
del model

In [ ]:
del outputs
gc.collect()
torch.cuda.empty_cache()

In [ ]:
outputs = model(**inputs)
p= torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu").tolist()

In [ ]:
!nvidia-smi

Tue Apr 19 20:52:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    60W / 149W |   1081MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print(predictions)

tensor([0.0314, 0.9075, 0.0611], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
#predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [ ]:
import torch

In [ ]:
predictions

In [ ]:
def get_sentiment(t):
    input = tokenizer.encode_plus(t, return_tensors='pt')
    p = mask_chunk(input, chunksize=64)
    c = input_id_chunk(input, chunksize=64)
    input_ids = torch.stack(c)
    attention_mask = torch.stack(p)
    input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()}
    outputs = model(**input_dict)
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    return probs.mean(dim=0)

In [ ]:
result = [get_sentiment(t) for t in finbert_list[0:100]]

Token indices sequence length is longer than the specified maximum sequence length for this model (1853 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import multiprocessing as mp

In [ ]:
def input_id_chunk(inputs,chunksize=128):
  input_id_chunks = inputs['input_ids'][0].split(chunksize-2)
  input_id_f = []
  for i in range(len(input_id_chunks)):
    id_chunk_size = input_id_chunks[i].shape[0]
    if (chunksize -2 - id_chunk_size) == 0:
      if i!=0:
        input_id_f.append(
          torch.cat([torch.tensor([101]),
                    input_id_chunks[i], 
                    torch.tensor([102])])
          )
      else:
        input_id_f.append(
            torch.cat([input_id_chunks[i], 
                      torch.tensor([102]),
                      torch.tensor([0])])
            )

    else:
      input_id_f.append(
          torch.cat([torch.tensor([101]),
                    input_id_chunks[i],
                    torch.Tensor([0] * (chunksize -1- id_chunk_size))])
          )
  return tuple(input_id_f)

def mask_chunk(inputs,chunksize=128):
  mask_chunks = inputs['attention_mask'][0].split(chunksize-2)
  mask_f = []
  for i in range(len(mask_chunks)):
    mask_chunks_size = mask_chunks[i].shape[0]
    if (chunksize -2 - mask_chunks_size) == 0:
      if i!=0:
        mask_f.append(
          torch.cat([torch.tensor([1]),
                    mask_chunks[i], 
                    torch.tensor([1])])
          )
      else:
        mask_f.append(
            torch.cat([mask_chunks[i], 
                      torch.tensor([1]),
                      torch.tensor([0])])
            )

    else:
      mask_f.append(
          torch.cat([torch.tensor([1]),
                    mask_chunks[i],
                    torch.Tensor([0] * (chunksize -1- mask_chunks_size))])
          )
  return tuple(mask_f)

In [ ]:
c = input_id_chunk(inputs)
for i in range(len(p)):
  print(len(p[i]))

128
128
128
128
128
128
104


In [ ]:
p = mask_chunk(tokens, chunksize=64)
c = input_id_chunk(tokens, chunksize=64)
input_ids = torch.stack(c)
attention_mask = torch.stack(p)

input_dict = {
    'input_ids': input_ids.long(),
    'attention_mask': attention_mask.int()
}
input_dict

{'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32),
 'input_ids': tensor([[  101,  2023,  3720,  ..., 26027,   102,     0],
         [  101,  2058,  1996,  ...,  4039,  7615,   102],
         [  101, 20643,  5446,  ..., 18755,  2813,   102],
         ...,
         [  101,  3745,  6599,  ...,  2105,  1996,   102],
         [  101, 19044,  3691,  ...,  2845,  5274,   102],
         [  101,  5924,  4012,  ...,     0,     0,     0]])}

In [ ]:
inputs

In [ ]:
%timeit model(**input_dict)

1 loop, best of 5: 5.71 s per loop


In [ ]:
%timeit model(**input_dict)

1 loop, best of 5: 5.59 s per loop


In [ ]:
outputs = model(**input_dict)
probs = torch.nn.functional.softmax(outputs[0], dim=-1)
probs = probs.mean(dim=0)
probs

tensor([0.0393, 0.0364, 0.9243], grad_fn=<MeanBackward1>)

In [ ]:
for i in range(len(p)):
  print(len(p[i]))

512
512


In [ ]:
for i in range(2):
  print(len(input_id_final[i]))

512
512


In [ ]:
input_id_final

In [ ]:
tuple([torch.cat([torch.tensor([1]),input_id_chunks[i]]) for i in range(len(input_id_chunks))])

In [ ]:
chunksize = 512
input_id_chunks = tokens['input_ids'][0].split(chunksize-2)
mask_chunks = tokens['attention_mask'][0].split(chunksize-2)

input_id_final = tuple([(torch.cat([torch.tensor([101]),
                                    input_id_chunks[i],
                                    torch.tensor([102]), 
                                    torch.Tensor([0] * (chunksize -2 - input_id_chunks[i].shape[0]))]))
                        if (chunksize - 2 - input_id_chunks[i].shape[0])>0 else 
                        torch.cat([torch.tensor([101]),input_id_chunks[i],torch.tensor([102])])
                        for i in range(len(input_id_chunks))
                        ])
mask_chunk_final = tuple([
                        (torch.cat([mask_chunks[i], torch.Tensor([0] * (chunksize - mask_chunks[i].shape[0]))]))
                        if (chunksize -2 - mask_chunks[i].shape[0])>0 else 
                        torch.cat([torch.tensor([1]),mask_chunks[i],torch.tensor([1])])
                        for i in range(len(mask_chunks))
                        ])
'''
for i in range(len(input_id_chunks)):
    # add CLS and SEP tokens to input IDs
    
    input_id_chunks[i] = torch.cat((
        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
    )])
    # add attention tokens to attention mask
    mask_chunks[i] = torch.cat([
        torch.tensor([1]), mask_chunks[i], torch.tensor([1])
    ])
    # get required padding length
    pad_len = chunksize - input_id_chunks[i].shape[0]
    # check if tensor length satisfies required chunk size
    if pad_len > 0:
        # if padding length is more than 0, we must add padding
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])
'''

'\nfor i in range(len(input_id_chunks)):\n    # add CLS and SEP tokens to input IDs\n    \n    input_id_chunks[i] = torch.cat((\n        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])\n    )])\n    # add attention tokens to attention mask\n    mask_chunks[i] = torch.cat([\n        torch.tensor([1]), mask_chunks[i], torch.tensor([1])\n    ])\n    # get required padding length\n    pad_len = chunksize - input_id_chunks[i].shape[0]\n    # check if tensor length satisfies required chunk size\n    if pad_len > 0:\n        # if padding length is more than 0, we must add padding\n        input_id_chunks[i] = torch.cat([\n            input_id_chunks[i], torch.Tensor([0] * pad_len)\n        ])\n        mask_chunks[i] = torch.cat([\n            mask_chunks[i], torch.Tensor([0] * pad_len)\n        ])\n'

In [ ]:
# get required padding length
pad_len = 512 - input_id_chunks[i].shape[0]
# check if tensor length satisfies required chunk size
if pad_len > 0:
    # if padding length is more than 0, we must add padding
    input_id_chunks[i] = torch.cat([
        input_id_chunks[i], torch.Tensor([0] * pad_len)
    ])
    mask_chunks[i] = torch.cat([
        mask_chunks[i], torch.Tensor([0] * pad_len)
    ])

NameError: ignored

In [ ]:
sentiment = []
chunk = [100,200,300,400,500]
a=0
for i in chunk:
  inputs = tokenizer(finbert_list[a:i], padding = True, truncation = True, return_tensors='pt')
  outputs = model(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  a=i
  sentiment.append(predictions)


In [ ]:
outputs = model(**inputs)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0307, 0.0773, 0.8920],
        [0.0998, 0.0709, 0.8293],
        [0.0934, 0.0329, 0.8737],
        [0.0511, 0.0252, 0.9238],
        [0.2247, 0.0207, 0.7547]], grad_fn=<SoftmaxBackward0>)


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df2.to_csv('AAPL_topics.csv')
!cp AAPL_topics.csv "drive/My Drive/Colab Notebooks/"

In [5]:
''' REGRESISON FOR TOPICS '''
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/Colab Notebooks/df_w_sentiment.csv'
df_bonus = pd.read_csv(path)

In [7]:
from sklearn.linear_model import LinearRegression

In [ ]:
df_bonus.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'date', 'title',
       'content', 'link', 'symbols', 'tags', 'clean', 'lemma', 'topics',
       'top_0', 'top_1', 'top_10', 'top_11', 'top_12', 'top_13', 'top_14',
       'top_15', 'top_16', 'top_17', 'top_18', 'top_19', 'top_2', 'top_20',
       'top_21', 'top_22', 'top_23', 'top_24', 'top_25', 'top_26', 'top_27',
       'top_28', 'top_29', 'top_3', 'top_4', 'top_5', 'top_6', 'top_7',
       'top_8', 'top_9', 'positive', 'negative', 'neutral'],
      dtype='object')

In [8]:
X = df_bonus[['top_0', 'top_1', 'top_10', 'top_11', 'top_12', 'top_13', 'top_14',
       'top_15', 'top_16', 'top_17', 'top_18', 'top_19', 'top_2', 'top_20',
       'top_21', 'top_22', 'top_23', 'top_24', 'top_25', 'top_26', 'top_27',
       'top_28', 'top_29', 'top_3', 'top_4', 'top_5', 'top_6', 'top_7',
       'top_8', 'top_9']]
y = df_bonus[['positive','negative',"neutral"]]
#y = (df_bonus["positive"]-df_bonus["negative"])*(1-df_bonus['neutral'])

In [ ]:
y1 = (df_bonus["positive"]-df_bonus["negative"])*(1-df_bonus['neutral'])
y2 = df_bonus["positive"]*df_bonus["positive"] - df_bonus["negative"]*df_bonus["negative"]
y3 = df_bonus["positive"] - df_bonus["negative"]

In [ ]:
y1

0      -0.102929
1      -0.044174
2      -0.102929
3      -0.153927
4       0.877479
          ...   
2995   -0.897718
2996    0.047872
2997    0.018644
2998   -0.922925
2999    0.011153
Length: 3000, dtype: float64

In [ ]:
y2

0      -0.102929
1      -0.044174
2      -0.102929
3      -0.153927
4       0.877479
          ...   
2995   -0.897718
2996    0.047872
2997    0.018644
2998   -0.922925
2999    0.011153
Length: 3000, dtype: float64

In [9]:
lm = LinearRegression(n_jobs=2).fit(X, y)

In [ ]:
lm.score(X,y)

0.23946113202182406

In [ ]:
np.mean(lm.coef_[0])

-2.402930398871694

In [ ]:
df_bonus[["top_1","top_2"]]

In [ ]:
df_bonus[df_bonus["top_2"]>0.8]

In [ ]:
df_bonus.loc[2995]

In [ ]:
lm.coef_

array([[-2.28050494, -2.3203838 , -2.36498883, -2.5306574 , -2.50753853,
        -2.21961906, -2.2435417 , -2.24993396, -2.38123089, -2.51039992,
        -2.4886355 , -2.41892828, -2.40185373, -2.51521049, -2.29546815,
        -2.41454423, -2.45230554, -2.51962005, -2.18053659, -2.42017399,
        -2.24513991, -2.32270299, -2.54050436, -2.56108177, -2.4594777 ,
        -2.39129597, -2.47135905, -2.33536581, -2.5335325 , -2.51137631],
       [-1.9201405 , -1.58090917, -2.00228208, -1.84314303, -1.87898892,
        -1.46384099, -1.89869455, -1.79554198, -1.54507871, -1.66830076,
        -2.02198406, -1.79227126, -1.86652547, -1.72691442, -1.70257816,
        -1.89624532, -1.76531286, -1.81168677, -1.75365495, -1.8279754 ,
        -1.33426   , -1.97993836, -1.66480222, -1.52666903, -1.94394448,
        -1.93618613, -1.71221883, -1.79641277, -1.67088069, -1.58542847],
       [ 4.20064543,  3.90129295,  4.36727089,  4.37380042,  4.38652742,
         3.68346002,  4.14223625,  4.04547592,  3

In [10]:
positive_score = lm.coef_[0] - np.mean(lm.coef_[0])

In [11]:
negative_score = lm.coef_[1] - np.mean(lm.coef_[1])

In [12]:
neutral_score = lm.coef_[1] - np.mean(lm.coef_[1])

In [13]:
top_list = ['top_0', 'top_1', 'top_10', 'top_11', 'top_12', 'top_13', 'top_14',
       'top_15', 'top_16', 'top_17', 'top_18', 'top_19', 'top_2', 'top_20',
       'top_21', 'top_22', 'top_23', 'top_24', 'top_25', 'top_26', 'top_27',
       'top_28', 'top_29', 'top_3', 'top_4', 'top_5', 'top_6', 'top_7',
       'top_8', 'top_9']

In [14]:
df = pd.DataFrame()

In [15]:
l=(positive_score - negative_score)*(1-neutral_score)
k = (df_bonus["positive"]-df_bonus["negative"])*(1-df_bonus['neutral'])
for i in range(30):
  df[f"{top_list[i]}_ap1"]=df_bonus[top_list[i]]*(k[i]+l[i])
  df[f"{top_list[i]}_pos"]=df_bonus[top_list[i]]*(df_bonus["positive"]+positive_score[i])
  df[f"{top_list[i]}_neg"]=df_bonus[top_list[i]]*(df_bonus["positive"]+negative_score[i])
  df[f"{top_list[i]}_neu"]=df_bonus[top_list[i]]*(df_bonus["positive"]+neutral_score[i])
  df[f"{top_list[i]}_average_adjusted"]=(df[f"{top_list[i]}_pos"]-df[f"{top_list[i]}_neg"])*(1-df[f"{top_list[i]}_neu"])
  df[f"{top_list[i]}_average"]=(df[f"{top_list[i]}_pos"]-df[f"{top_list[i]}_neg"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To

In [16]:
df

,top_0_ap1,top_0_pos,top_0_neg,top_0_neu,top_0_average_adjusted,top_0_average,top_1_ap1,top_1_pos,top_1_neg,top_1_neu,...,top_8_neg,top_8_neu,top_8_average_adjusted,top_8_average,top_9_ap1,top_9_pos,top_9_neg,top_9_neu,top_9_average_adjusted,top_9_average
0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,-0.0,-0.0,-0.008594,-0.002669,0.007147,0.007147,-0.009745,-0.009815
2,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,-0.0,-0.0,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
2996,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2997,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2998,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.0,0.0,-0.0,-0.0,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000


In [17]:
df_bonus.merge(df, right_index=True, left_index=True)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,title,content,link,symbols,tags,clean,...,top_8_neg,top_8_neu,top_8_average_adjusted,top_8_average,top_9_ap1,top_9_pos,top_9_neg,top_9_neu,top_9_average_adjusted,top_9_average
0,0,0,0,2022-02-28T13:07:17+00:00,Dow Jones Futures Pare 'Swift' Market Losses; ...,Dow Jones futures signal a market rally revers...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'NASDAQ FUTURES'...",Dow Jones futures signal market rally reversal...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,2022-02-28T10:12:12+00:00,Corporate America condemns Russia — what that ...,This article first appeared in the Morning Bri...,https://finance.yahoo.com/news/corporate-ameri...,"['TSLA.US', 'AAPL.US', 'CRM.US', 'ROKU.US', 'S...","['RUSSIA', 'MULTINATIONAL COMPANIES', 'UKRAINE']",This article first appeared the Morning Brief ...,...,0.0,0.0,-0.0,-0.0,-0.008594,-0.002669,0.007147,0.007147,-0.009745,-0.009815
2,2,2,2,2022-02-27T23:22:17+00:00,Dow Jones Futures Signal 'Swift' Market Revers...,Dow Jones futures signal a market rally revers...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'RUSSIA', 'DOW J...",Dow Jones futures signal market rally reversal...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,3,2022-02-27T18:18:17+00:00,Dow Jones Futures Loom: Watch Out For 'Swift' ...,The West ramps up sanctions vs. Russia amid th...,https://finance.yahoo.com/m/5e9706a2-d66c-3d05...,"['TSLA.US', 'AAPL.US', 'MSFT.US', 'REGN.US', '...","['DOW JONES', 'STOCK MARKET', 'NASDAQ FUTURES'...",The West ramps sanctions Russia amid the ongoi...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,4,2022-02-27T16:53:33+00:00,Berkshire Hathaway Earnings: What Happened wit...,Berkshire Hathaway (BRK.A) beat estimates for ...,https://finance.yahoo.com/m/a3289340-767e-3109...,"['BRK-A.US', 'BRK-B.US', 'AAPL.US', 'AAPL34.SA...","['WARREN BUFFETT', 'BERKSHIRE HATHAWAY', 'EARN...",Berkshire Hathaway BRK beat estimates for earn...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,2995,2995,2021-10-29T15:19:00+00:00,Why Apple bulls aren’t sweating the latest ear...,"Apple shares are off Friday, but analysts say ...",https://finance.yahoo.com/m/7b140280-345e-3fb0...,"['AAPL.US', 'AAPL34.SA', 'AAPL.MX', 'APC.F', '...","['APPLE INC', 'PRICE TARGET', 'WALL STREET ANA...",Apple shares are off Friday but analysts say b...,...,0.0,0.0,-0.0,-0.0,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
2996,2996,2996,2996,2021-10-29T15:16:03+00:00,"Apple (AAPL) Q4 Earnings In Line, Chip Shortag...",Apple AAPL reported fourth-quarter fiscal 2021...,https://finance.yahoo.com/news/apple-aapl-q4-e...,"['AAPL.US', 'AAPL34.SA', 'AAPL.MX', 'APC.F', '...","['APPLE', 'APPLE WATCH', 'APPLE INC', 'REPORTE...",Apple AAPL reported fourth-quarter fiscal 2021...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2997,2997,2997,2997,2021-10-29T15:15:15+00:00,Steve Wozniak on accepting crypto pitches in ‘...,Yahoo Finance’s Brian Sozzi and Julie Hyman sp...,https://finance.yahoo.com/video/steve-wozniak-...,"['AAPL.US', 'FB.US', 'BTC-USD.CC', 'BTC.CC', '...","['JULIE HYMAN', 'BRIAN SOZZI', 'STEVE WOZNIAK']",Yahoo Finance Brian Sozzi and Julie Hyman spea...,...,0.0,0.0,0.0,0.0,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2998,2998,2998,2998,2021-10-29T15:06:00+00:00,"Apple’s Troubles Are About Supply, Not Demand....",The company would have crushed revenue and pro...,https://finance.yahoo.com/m/90a19341-9dda-3cad...,"['AAPL.US', 'AAPL34.SA', 'AAPL.MX', 'APC.F', '...",['APPLE'],The company would have crushed revenue and pro...,...,0.0,0.0,-0.0,-0.0,-0.00000

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 43.8 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15192 sha256=e0df87023943e185f2c127464d4ec5bd27493530edbe0f14b884ade2c447b699
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [4]:
''' Get market data from yfinance '''
import yfinance as yf
from yahoofinancials import YahooFinancials

In [ ]:
df_bonus['date'][2999][:10]

'2021-10-29'

In [20]:
aapl_df = yf.download('AAPL', 
                      start=df_bonus['date'][len(df_bonus)-1][:10], 
                      end=add_days_to_date(date=df_bonus['date'][0][:10], days=2), 
                      progress=True,
)
aapl_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-29,147.220001,149.940002,146.410004,149.800003,149.172195,124953200
2021-11-01,148.990005,149.699997,147.800003,148.960007,148.335709,74588300
2021-11-02,148.660004,151.570007,148.649994,150.020004,149.391266,69122000
2021-11-03,150.389999,151.970001,149.820007,151.490005,150.855103,54511500
2021-11-04,151.580002,152.429993,150.639999,150.960007,150.327316,60394600
...,...,...,...,...,...,...
2022-02-23,165.539993,166.149994,159.750000,160.070007,159.835175,90009200
2022-02-24,152.580002,162.850006,152.000000,162.740005,162.501251,141147500
2022-02-25,163.839996,165.119995,160.869995,164.850006,164.608154,91974200


In [19]:
from datetime import timedelta
def subtract_days_from_date(date, days):
    subtracted_date = pd.to_datetime(date) - timedelta(days=days)
    subtracted_date = subtracted_date.strftime("%Y-%m-%d")

    return subtracted_date
  
def add_days_to_date(date, days):
    added_date = pd.to_datetime(date) + timedelta(days=days)
    added_date = added_date.strftime("%Y-%m-%d")

    return added_date

In [ ]:
add_days_to_date(date=df_bonus['date'][0][:10], days=1)

'2022-03-01'

In [ ]:
aapl_df.index

DatetimeIndex(['2021-10-29', '2021-11-01', '2021-11-02', '2021-11-03',
               '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09',
               '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15',
               '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
               '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-26',
               '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02',
               '2021-12-03', '2021-12-06', '2021-12-07', '2021-12-08',
               '2021-12-09', '2021-12-10', '2021-12-13', '2021-12-14',
               '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-20',
               '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-27',
               '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
               '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06',
               '2022-01-07', '2022-01-10', '2022-01-11', '2022-01-12',
               '2022-01-13', '2022-01-14', '2022-01-18', '2022-01-19',
      

In [ ]:
aapl_df[np.logical_and(("2021-11-04">aapl_df.index), (aapl_df.index>"2021-10-29"))]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-11-01,148.990005,149.699997,147.800003,148.960007,148.553650,74588300
2021-11-02,148.660004,151.570007,148.649994,150.020004,149.610764,69122000
2021-11-03,150.389999,151.970001,149.820007,151.490005,151.076767,54511500
